# BTC Solar Solver


In [25]:
# PV System Info
pv_system = {'location':{
		'lat':38.897957,
		'lon':-77.036560},
	'specs':{
		'array_type':1, # Fixed axis
		'azimuth':180,
		'tilt':28.7,
		'losses':1}}

wh_solar = SolarSimulator(pv_system)
wh_solar.simulate_solar()


Starting data download for solar using 1 thread workers.
File already exists. Skipping download: /home/carlos/Documents/GitHub/BTCsolar/data/PySAM Downloaded Weather Files/nsrdb_38.897957_-77.03656_psm3-tmy_60_tmy.csv


{'spec': 'r7|7',
 'shape':       Month  Day  Hour  solar
 0         1    1     0    0.0
 1         1    1     1    0.0
 2         1    1     2    0.0
 3         1    1     3    0.0
 4         1    1     4    0.0
 ...     ...  ...   ...    ...
 8755     12   31    19    0.0
 8756     12   31    20    0.0
 8757     12   31    21    0.0
 8758     12   31    22    0.0
 8759     12   31    23    0.0
 
 [8760 rows x 4 columns]}